# JuliaDB + OnlineStats

[JuliaDB](https://github.com/JuliaComputing/JuliaDB.jl) is a package for working with large persistent datasets.  The online/parallel algorithms in OnlineStats work efficiently with `reduce` operations in JuliaDB to calculate statistics and models out-of-core or in parallel.

In [1]:
using JuliaDB, OnlineStats

## Basics

#### The `reduce` function can accept Julia functions and `OnlineStat`s as reducers.

In [2]:
t = table(@NT(x = randn(100), y = randn(100)))

@show reduce(+, t, select = :x)

reduce(Sum(), t, select = :x)

reduce(+, t, select=:x) = 2.45258956707799


▦ Series{0}  |  EqualWeight  |  nobs = 100
└── Sum{Float64}(2.45259)

#### If the selection is multiple columns, the `OnlineStat` must be able to handle inputs of that size.  

For example, a `Mean` can't accept two numbers as an input, so this errors:

In [3]:
reduce(Mean(), t; select = (:x, :y))

LoadError: [91mMethodError: no method matching fit!(::OnlineStats.Series{0,Tuple{OnlineStats.Mean},OnlineStatsBase.EqualWeight}, ::NamedTuples._NT_x_y{Float64,Float64})[0m
Closest candidates are:
  fit!([91m::OnlineStats.CovMatrix[39m, ::Union{AbstractArray{T,1} where T, NamedTuples.NamedTuple, Tuple}, [91m::Float64[39m) at /Users/joshday/.julia/v0.6/OnlineStats/src/stats/stats.jl:171
  fit!([91m::OnlineStats.HyperLogLog[39m, ::Any, [91m::Float64[39m) at /Users/joshday/.julia/v0.6/OnlineStats/src/stats/stats.jl:313
  fit!([91m::OnlineStats.LinReg[39m, ::Union{AbstractArray{T,1} where T, NamedTuples.NamedTuple, Tuple}, [91m::Real[39m, [91m::Float64[39m) at /Users/joshday/.julia/v0.6/OnlineStats/src/stats/linregbuilder.jl:37
  ...[39m

#### We can, however, create an object that calculates the mean for each column:

In [4]:
s = reduce(2Mean(), t, select = (:x, :y))

▦ Series{1}  |  EqualWeight  |  nobs = 100
└── MV{Mean}(0.02452589567077987, -0.03142483715376148)

# Partitions

The `Partition` type is designed around visualizing large datasets. It incrementally partitions the data into equal parts, each part summarized by an `OnlineStat`.

In [5]:
using Plots; gr()

Plots.GRBackend()

In [6]:
# x = randn()
# s = Series(Partition(Extrema(), 40))

# @gif for i in 1:2_000
#     plot(fit!(s, x += randn()))
# end every 100;

![](tmp.gif)

## Partition on the `diamonds` dataset

In [7]:
diamonds = loadtable("diamonds.csv"; indexcols = [:carat])

Table with 53940 rows, 10 columns:
carat  cut          color  clarity  depth  table  price  x      y      z
───────────────────────────────────────────────────────────────────────────
0.2    "Premium"    "E"    "SI2"    60.2   62.0   345    3.79   3.75   2.27
0.2    "Premium"    "E"    "VS2"    59.8   62.0   367    3.79   3.77   2.26
0.2    "Premium"    "E"    "VS2"    59.0   60.0   367    3.81   3.78   2.24
0.2    "Premium"    "E"    "VS2"    61.1   59.0   367    3.81   3.78   2.32
0.2    "Premium"    "E"    "VS2"    59.7   62.0   367    3.84   3.8    2.28
0.2    "Ideal"      "E"    "VS2"    59.7   55.0   367    3.86   3.84   2.3
0.2    "Premium"    "F"    "VS2"    62.6   59.0   367    3.73   3.71   2.33
0.2    "Ideal"      "D"    "VS2"    61.5   57.0   367    3.81   3.77   2.33
0.2    "Very Good"  "E"    "VS2"    63.4   59.0   367    3.74   3.71   2.36
0.2    "Ideal"      "E"    "VS2"    62.2   57.0   367    3.76   3.73   2.33
0.2    "Premium"    "D"    "VS2"    62.3   60.0   367    

In [11]:
plot(diamonds, :carat, reducer=Extrema())

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 carat 
 
 
 Nobs 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Value of Extrema 
 
 
 
 Parts

In [9]:
reduce(@NT(x = Series(Mean(), Variance())), diamonds; select = :carat)

(x = ▦ Series{0}  |  EqualWeight  |  nobs = 53940
├── Mean(0.79794)
└── Variance(0.224687))